In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [ ]:
import torch
torch.cuda.is_available()

In [ ]:
import transformers
import torch
import torchvision
import torchaudio
import numpy as np
import pandas as pd
import moviepy
import librosa
import glob
import moviepy.editor as mp
from torch.utils.data import TensorDataset, DataLoader, random_split
import librosa.display
import matplotlib.pyplot as plt
import tarfile
import matplotlib
import tqdm
import PIL
import cv2 as cv
import os
from os import walk, listdir
import shutil
from tqdm import tqdm
import pickle
from transformers import AutoTokenizer, BertForTokenClassification, BertForSequenceClassification,BertPreTrainedModel, BertModel, BertTokenizer, AutoModel
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
import torch.nn.functional as F
from scipy.io.wavfile import read
import torchvision.models as models
import torchvision.transforms as transforms
import torch.utils.data as data
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.metrics import accuracy_score
from transformers import ViTFeatureExtractor, ViTModel
import requests

EXTRACTING FEATURES FROM IMAGE FRAMES

In [ ]:
!pip install --upgrade transformers

In [ ]:
FOLDER_NAME = '/path/to/folder/containing/image frames'
CATEGORY = 'Explicit Hate Videos' #change to Implicit Hate Videos, Non Hate Videos as needed
FEATURE_DIR = '/path/to/folder/for/storing/features'

In [ ]:
from transformers import ViTModel
import torch
import random

# Initialize model
model = ViTModel.from_pretrained("google/vit-base-patch16-224-in21k")

In [ ]:
from PIL import Image

In [ ]:
from transformers import ViTImageProcessor
image_processor = ViTImageProcessor.from_pretrained("google/vit-base-patch16-224-in21k")

In [ ]:
# Initialize a list to store the pooler outputs
last_hidden_states = []
folders =[]
minFrames = 100

df = pd.read_excel(f'{FOLDER_NAME}/{CATEGORY}.xlsx')
cnt = 1

# Loop through each folder
for folder in (df['Video_ID']):
    frames = []  # Store frame embeddings for the current folder
    folder_frames = os.listdir(FOLDER_NAME + folder)
    num_frames = len(folder_frames)
    
    if num_frames <= 100:
        selected_frames = folder_frames
    else:
        # Uniformly sample 100 frames
        step = int(num_frames/minFrames)
        indices = list(range(0, num_frames, step))
        selected_frames = [folder_frames[i] for i in indices]
        selected_frames = selected_frames[:100]
        
    for image in selected_frames:
        img = Image.open(FOLDER_NAME + folder+'/'+image)
        transform = transforms.Compose([
            transforms.Resize((224, 224)), 
            transforms.ToTensor(),  
            transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
        ])
        input_tensor = transform(img).unsqueeze(0)  

        with torch.no_grad():
            outputs = model(pixel_values=input_tensor)
        last_hidden_state = outputs.last_hidden_state.detach().numpy()
        frames.append(last_hidden_state)
    # If the folder has less than 100 frames, append zeroes
    while len(frames) < 100:
        frames.append(np.zeros_like(frames[0], dtype=np.float32))
    # Stack the frames along the first dimension to get shape (100, 768)
    last_hidden_states.append(frames)
    print(len(last_hidden_states), len(frames))

arr_img = np.array(last_hidden_states)
np.save(f"{FEATURE_DIR}/{CATEGORY}_img_2Dfeatures.npy",arr_img)